In [ ]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
from urllib.parse import quote, urlencode
import time

In [ ]:
API_key = "INSERT_API_KEY_HERE"
SECRET = "INSERT_SECRET_KEY_HERE"
url = "https://api.idealista.com/oauth/token"

### Steps:
- URL encode your API key and secret according to RFC 1738
- Concatenate the encoded like this "API:Secret" into a single string
- Base64 encode the string from the previous step
- Use this string to get the bearer token to use for the search

In [ ]:
def get_oauth_token():
    url = "https://api.idealista.com/oauth/token"
    apikey= API_key #sent by idealista
    secret= SECRET  #sent by idealista
    encoded_api_key = quote(apikey, safe='')
    encoded_api_secret = quote(secret, safe='')
    enc = f"{encoded_api_key}:{encoded_api_secret}"
    auth = base64.b64encode(enc.encode())
    headers = {'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8' ,'Authorization' : 'Basic ' + auth.decode()}
    params = urlencode({'grant_type':'client_credentials'})
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [ ]:
def search_api(token, url):
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

Here we search for each county using the code that can be seen in this list, that we got by reading the HTML code in the website and reading the API documentation. We chose those counties based on the amount of listing we could get to form a reliable sample that could fit within the limits of our alloted api access.

In [ ]:
concelhos = {"lisboa": [176, '0-EU-PT-11-06'], "cascais": [73, '0-EU-PT-11-05'], "amadora": [19, '0-EU-PT-11-15'],"oeiras" : [30,'0-EU-PT-11-10'] , "odivelas" : [17,'0-EU-PT-11-16'], "loures" : [26,'0-EU-PT-11-07'], "VF_Xira" : [17,'0-EU-PT-11-14']}

for key, value in concelhos.items():
    df_tot = pd.DataFrame()
    for i in range(value[0]):
        pagina = str(i)


        locationId = value[1] #lisboa
        country = 'pt' #values: es, it, pt
        language = 'pt' #
        max_items = '50'
        operation = 'sale'
        property_type = 'homes'
        order = 'publicationDate'
        sort = 'desc'
        #specific to houses:
        #preservation= 'renew'
        url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+
                   '&maxItems='+max_items+
                   '&order='+order+
                   '&locationId='+locationId+
                   '&propertyType='+property_type+
                   '&sort='+sort+
                   '&numPage='+pagina+
                   '&language='+language)

        oauth_token = get_oauth_token()
        data = search_api(oauth_token, url)
        df = pd.DataFrame.from_dict(data['elementList'])
        df_tot = pd.concat([df_tot,df])
        time.sleep(1)
    df_tot.to_csv(key+".csv", index=False)

In [ ]:
df_tot.head()

,propertyCode,thumbnail,numPhotos,floor,price,priceInfo,propertyType,operation,size,rooms,...,has3DTour,has360,hasStaging,topNewDevelopment,topPlus,externalReference,district,description,highlight,newDevelopmentFinished
0,33418366,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,21,1,330000.0,{'price': {'amount': 330000.0}},flat,sale,150.0,3,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
1,33418269,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,14,NaN,170000.0,{'price': {'amount': 170000.0}},flat,sale,80.0,2,...,False,False,False,False,False,VFX070,NaN,NaN,NaN,NaN
2,33418175,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,20,NaN,169900.0,{'price': {'amount': 169900.0}},flat,sale,91.0,3,...,False,False,False,False,False,122601320-80,NaN,NaN,NaN,NaN
3,33418110,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,27,NaN,180000.0,{'price': {'amount': 180000.0}},flat,sale,77.0,2,...,False,False,False,False,False,120992531-302,NaN,NaN,NaN,NaN
4,33417855,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,17,NaN,140000.0,{'price': {'amount': 140000.0}},flat,sale,62.0,2,...,False,False,False,False,False,C0483-00706,NaN,NaN,NaN,NaN
